In [ ]:
#| echo: false
#| output: false
%load_ext autoreload
%autoreload 2

::: {style="font-size: 50%;"}
## Consideraciones previas

- Se realizará una breve introducción a modelos de aprendizaje automático, pero se recomienda tener algún conocimiento previo para seguir el taller. 

- Durante el seminario se utilizarán los siguientes paquetes (**python**):

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Imports"
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, to_rgb
from great_tables import GT, from_column, style, loc
from collections import Counter

# from sklearn import datasets
# Modelado
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score,
    roc_curve,
    RocCurveDisplay,
    log_loss,
    recall_score,
    brier_score_loss,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from IPython.display import display, Markdown

# Funciones adicionales: 
from custom_functions import (
    plot_distribution,
    plot_calibration_models,
    plot_calibration,
    calibrate_model,
    calibration_table,
    calculate_clf_metrics,
)

import sklearn
sklearn.set_config(transform_output="pandas")

In [ ]:
#| echo: false
import matplotlib
import great_tables
from IPython.display import display, Markdown, Latex
Markdown(f"""
**Procesamiento de datos**:\n
📦 pandas=={pd.__version__}\n
📦 numpy=={np.__version__}\n
\n
**Modelado**:\n
📦 scikit-learn=={sklearn.__version__}\n
\n
**Visualización y tablas:**\n
📦 matplotlib=={matplotlib.__version__}\n
📦 seaborn=={sns.__version__}\n
📦 great_tables=={great_tables.__version__}\n
""")

In [ ]:
#| echo: false
#| code-fold: false
color_verde = "#255255"
color_verde_claro = "#BDCBCC"

:::


# Introducción a machine learning

## Machine Learning

En general al hablar de machine learning se hace referencia a diversos tipos de modelos. En esta presentación se analizará el caso particular de los modelos de clasificación, haciendo énfasis en la estimación de probabilidad.

```{mermaid}
flowchart LR
    ml[Machine Learning]
    supervised[Aprendizaje <br> supervisado]
    unsupervised[Aprendizaje <br> no supervisado]
    semisupervised[Aprendizaje <br> semi supervizado]
    reinforce[Aprendizaje <br> por refuerzo]

    ml_clf[Modelos de <br>clasificación]
    ml_reg[Modelos de <br>regresión]
    ml_clfclass[Predicción de<br>clase]
    ml_clfprob[Predicción de <br>probabilidad]:::redclass

    ml-->supervised
    ml-->unsupervised
    ml-->semisupervised
    ml-->reinforce

    supervised-->ml_clf
    supervised-->ml_reg
    ml_clf-->ml_clfclass
    ml_clf-->ml_clfprob

    classDef redclass fill:#255255,stroke:#333,stroke-width:2px,color:white;
    classDef blueclass fill:#BDCBCC,stroke:#333,stroke-width:2px,color:white;
```


## ¿Qué es un modelo de clasificación binaria?
::: {style="font-size: 50%;"}

Se busca predecir la [probabilidad de ocurrencia]{style="color: blue"} de un evento a partir de ciertas características observables:

> $P(y=1) = f(X)$
>
> *Siendo y una variable que puede tomar 2 valores: 0 o 1*

::: {.fragment .fade-in}
<br> 
**Ejemplos de clasificación:**

▪️**Iris**: Clasificación de especies de plantas

▪️**Titanic**: Clasificación de individuos en sobrevivientes y no sobrevivientes

::: {.fragment .highlight-red}
▪️**German Credit**: Clasificación de individuos en riesgosos o no riesgosos
:::
:::
:::


# Caso: German Credit

## Datos

::: {style="font-size: 50%;"}

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Código"
PATH_DATA = 'https://raw.githubusercontent.com/ayseceyda/german-credit-gini-analysis/master/gc.csv'
TARGET = "Risk"

df = (pd.read_csv(PATH_DATA)
  .drop('Unnamed: 0', axis=1)
  .assign(Risk = lambda x: np.where(x['Risk']=='good',0,1))
)

Se cuenta con un dataset de `{python} df.shape[0]` observaciones y `{python} df.shape[1]` variables.

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Código"
#| tbl-cap: "Datos de German Credit (muestra de 6 observaciones)"
#| label: tbl-credit-data
def map_color(data):
    return (data[TARGET] == 1).map(
        {True: color_verde_claro, False: 'white'}
    )

(GT(df.sample(6, random_state=123)
        .set_index(TARGET)
        .reset_index()
    )
    .fmt_currency(columns="Credit amount")
    .tab_style(
        style=style.fill(color=map_color), locations=loc.body(columns=df.columns.tolist()),
    )
    .tab_style(
        style=style.text(color='red', weight = "bold"), locations=loc.body(TARGET),
    )
    .tab_options(
        column_labels_background_color=color_verde,
        table_font_names="Times New Roman"
    )
)

La variable objetivo (target) es Risk, donde el porcentaje de observaciones de clase 1 (riesgosos) es `{python} f"{df['Risk'].sum()/df.shape[0]:.1%}"`

:::

## Particiones
::: {style="font-size: 50%;"}

Partición en dataset de entrenamiento, validación y evaluación.

- Dataset de entrenamiento --> Ajuste del modelo
- Dataset de validación --> Calibración del modelo
- Dataset de evaluación --> Métricas del modelo calibrado

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Código"
#| label: particiones
y = df[TARGET]
X = df.drop([TARGET], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, shuffle=True, stratify=y, random_state=42
)

# Lo ideal es utilizar la partición de validación para calibración
X_valid, X_test, y_valid, y_test = train_test_split(
    X_test, y_test, test_size=0.5, shuffle=True, stratify=y_test, random_state=42
)

print(f"N observaciones en entrenamiento: {X_train.shape[0]}")
print(f"N observaciones en validación: {X_valid.shape[0]}")
print(f"N observaciones en evaluación: {X_test.shape[0]}")

::: 

# Modelo simple

::: {style="font-size: 50%;"}
## Modelo de clasificación simple

Se consideran 2 variables para ajustar un modelo de arbol de decisión con máxima profundidad=2

$$
P(\text{Risk}=1) = f(\text{Credit amount}, \text{Age})
$$

::: columns
::: {.column width="50%"}
::: {.fragment .fade-in}

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Código"
#| tbl-cap: "Datos de German Credit (2 variables)"
#| label: tbl-credit-data-1var
subset_cols = ['Age', 'Credit amount']

(GT(pd.concat([y_train, X_train], axis=1)
        .sample(6, random_state=123)
        .set_index(TARGET)
        [subset_cols]
        .reset_index()
    )
    .fmt_currency(columns="Credit amount")
    .tab_style(
        style=style.fill(color=map_color), locations=loc.body(columns=df.columns.tolist()),
    )
    .tab_style(
        style=style.text(color='red', weight = "bold"), locations=loc.body(TARGET),
    )
    .tab_options(
        column_labels_background_color=color_verde,
        table_font_names="Times New Roman"
    )
)

Ajuste del modelo:

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Código"
X_train_subset = X_train[subset_cols].copy()
clf = DecisionTreeClassifier(max_depth=2).fit(X_train_subset, y_train)
clf

:::
:::

::: {.column width="50%"}
::: {.fragment .fade-in}

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Código"
#| layout-align: center
#| fig-cap: "Modelo de árbol de decisión"
#| label: fig-credit-tree
colors = [color_verde, 'red']
labels = ['No riesgoso','Riesgoso']
plt.figure(figsize=(6,6))
arbol = plot_tree(
    clf,
    filled=True,
    impurity=False,
    feature_names=X_train_subset.columns.tolist(),
    class_names=labels,
    fontsize=8
)
for i, impurity, value in zip(arbol, clf.tree_.impurity, clf.tree_.value):
    # let the max value decide the color; whiten the color depending on impurity (gini)
    r, g, b = to_rgb(colors[np.argmax(value)])
    f = impurity * 2 # for N colors: f = impurity * N/(N-1) if N>1 else 0
    i.get_bbox_patch().set_facecolor((f + (1-f)*r, f + (1-f)*g, f + (1-f)*b))
    i.get_bbox_patch().set_edgecolor('black')
plt.show()

:::
:::
:::
:::

::: {style="font-size: 50%;"}
## Evaluación de un modelo de clasificación
::: columns
::: {.column width="50%"}
::: {.fragment .fade-in}
Se utiliza el método `clf.predict()`para predecir la clase (0,1) según el modelo (`clf`). A partir de la clase predicha se obtiene la matriz de confusión:

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Ver código"
#| label: tbl-cm
#| tbl-cap: 'Matriz de confusión'
preds = pd.DataFrame({'Valor observado':y_test,'Valor predicho':clf.predict(X_test[subset_cols])})
display(GT(preds
    .groupby(['Valor observado','Valor predicho'], as_index=False).size()
    .pivot(index='Valor observado', columns='Valor predicho', values='size')
    .rename({0:'0',1:'1'}, axis=1)
    .reset_index()
    )
    .tab_spanner('Valor predicho', columns = ['0','1'])
    .data_color(
        columns=['0','1'],
        domain=[0,X.shape[0]],
        palette=[color_verde_claro, 'red'],
        na_color="white",
    )
)

:::
:::
::: {.column width="50%"}
::: {.fragment .fade-in}

Además de predecir una clase, sklearn permite predecir una "probabilidad" con `clf.predict_proba()`:

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Ver código"
#| label: tbl-predictproba
#| tbl-cap: 'clf.predict_proba(), muestra de 2 observaciones'
y_pred_proba = clf.predict_proba(X_test[subset_cols])[:, 1]
preds = pd.DataFrame({
    'y_obs':y_test,
    'predict_proba':y_pred_proba
})
GT(preds.sample(2, random_state=42)).fmt_number('predict_proba')

In [ ]:
metrica_auc = roc_auc_score(y_true=y_test, y_score=y_pred_proba)
metrica_logloss = log_loss(y_true=y_test, y_pred=y_pred_proba)
metrica_brierloss = brier_score_loss(y_true=y_test, y_prob=y_pred_proba)

print(f"""
ROC AUC = {metrica_auc:.2}, Log loss = {metrica_logloss:.2}, Brier loss = {metrica_brierloss:.2}
""")

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Ver código"
#| label: fig-distrib-tree
#| fig-cap: 'Distribución de "probabilidad" predicha'
distrib=plot_distribution(data=preds, pred_column='predict_proba', class_0_color=color_verde)
distrib

:::
:::
:::
:::

# Modelos más complejos

::: {style="font-size: 50%;"}
## Preprocesamiento

Se construye un pipeline de preprocesamiento de variables, diferenciando el procesamiento según tipo de datos: 

- Variables categóricas
- Variables numéricas

<br>

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Ver código"
numeric_transformer = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline([
    ('ohe',OneHotEncoder(
        min_frequency=0.05,
        handle_unknown='infrequent_if_exist',
        sparse_output=False)
    )
])

preproc = ColumnTransformer([
    ('num', numeric_transformer,
      make_column_selector(dtype_include=['float','int'])),
    ('cat', categorical_transformer,
      make_column_selector(dtype_include=['object','category']))
], verbose_feature_names_out=False)

In [ ]:
#| echo: false
#| code-fold: false
#| classes: custom_class_html_display
#| layout-align: center
preproc

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Código"
#| tbl-cap: "Datos de German Credit preprocesados (muestra de 2 observaciones)"
#| label: tbl-credit-data-preproc
display(GT(preproc.fit_transform(X_train).sample(2).round(2))
    .tab_options(
        column_labels_background_color=color_verde,
        table_font_names="Times New Roman"
    )
)

::: 


## Modelado
::: {style="font-size: 50%;"}

Se ajustan distintos tipos de modelos reutilizando el mismo `pipeline` de preprocesamiento de datos:

In [ ]:
def gen_pipe(model):
    pipe = Pipeline([
        ('preproc', preproc),
        ('model',model)
    ])
    return pipe

pipe_hist = gen_pipe(
    model = HistGradientBoostingClassifier(
        max_depth=4,
        learning_rate=0.1,
        max_iter=1000,
        random_state=42,
    )
)
pipe_hist_balanced = gen_pipe(
    model = HistGradientBoostingClassifier(
        max_depth=4,
        learning_rate=0.1,
        max_iter=1000,
        random_state=42,
        class_weight='balanced'
    )
)

pipe_tree = gen_pipe(
    model = DecisionTreeClassifier(random_state=42)
)
pipe_tree = gen_pipe(
    model = DecisionTreeClassifier(random_state=42)
)
pipe_reglog = gen_pipe(
    model = LogisticRegression(random_state=42)
)
pipe_reglog_balanced = gen_pipe(
    model = LogisticRegression(random_state=42, class_weight='balanced')
)
pipe_svc = gen_pipe(model = SVC(random_state=42, probability=True))

models_dict = {
    'Hist gradient boosting': pipe_hist,
    'Decision tree': pipe_tree,
    'Logistc regression': pipe_reglog,
    'SVC': pipe_svc,
    'Logistc regression (balanced)': pipe_reglog_balanced,
    'Hist gradient boosting (balanced)': pipe_hist_balanced
}

::: {.nav-pills}
::: {.panel-tabset}

In [ ]:
# | output: asis
for i in models_dict.keys():
    display(Markdown(f"## {i}"))
    models_dict.get(i).fit(X_train, y_train)
    display(models_dict.get(i))
    display(Markdown(f" "))

:::
:::
:::

::: {style="font-size: 50%;"}
## Evaluación

Se calculan las métricas de cada uno de los modelos:

In [ ]:
#| tbl-cap: "Métricas de los modelos en la partición de evaluación"
#| label: tbl-metrics-uncalibrated
metrics_df = pd.DataFrame()
for i in models_dict.keys():
    pipe = models_dict.get(i)
    preds = pd.DataFrame({
        'y_true': y_test,
        'y_pred': pipe.predict(X_test),
        'y_pred_prob': pipe.predict_proba(X_test)[:,1],
    })
    m = calculate_clf_metrics(
        preds['y_true'], preds['y_pred'], preds['y_pred_prob'], 
        name=i
    ).reset_index().rename({'index':'Modelo'},axis=1)
    metrics_df = pd.concat([metrics_df, m], axis=0)

(GT(metrics_df.round(2))
    .data_color(
        columns=['Accuracy','Precision','Recall','F1','ROC AUC'],
        palette=[color_verde_claro, color_verde]
    )
    .data_color(
        columns=['Log Loss','Brier Loss'], palette=['white', 'red']
    )
)

:::

::: {style="font-size: 50%;"}
## Problema

Planteo de problema organizacional

:::

# Calibración de probabilidades

::: {style="font-size: 50%;"}
## Calibración de probabilidades

**Objetivo:** 

Se busca encontrar una función que ajuste la relación entre los scores predichos por el modelo (predict_proba) y las probabilidades reales

> $P(y_i=1)= f(z)$
>
> Siendo: $P(y_{i}=1)$ la probabilidad calibrada para el individuo $i$ y $z_{i}$ el output del modelo no calibrado (score)

Un clasificador binario bien calibrado debería clasificar de forma tal que para las observaciones que predice un score (predict_proba) = 0.8, se espera que un 80% de las observaciones correspondan a la clase positiva (1).

**Referencias:**

- [Calibración de probabilidades en {scikit-learn} 📦 ](https://scikit-learn.org/stable/modules/calibration.html)

:::

::: {style="font-size: 50%;"}
## Calibración de probabilidades (Cont.)

::: {.fragment .fade-in}
**Métodos de calibración**

::: {.fragment .highlight-red}
 Calibración Sigmoide (Platt scaling)

 Calibración Isotónica
:::

 Calibración Beta (Nuevo)

 Calibración Spline (Nuevo)
:::
:::

::: {style="font-size: 50%;"}
## Calibración sigmoide (Platt scaling)

Este método asume una relación logística entre los scores (z) y la probabilidad real (p):

> $log(\frac{p}{1-p})=\alpha+\beta(z_{i})$ 
>
>
> $P(y_{i}=1) = \frac{1}{1+(e^{-(α+β(z_{i}))})}$
>
>
> Siendo: $y_{i}$ el valor observado para el individuo $i$ y $z_{i}$ el output del modelo no calibrado

<br>
Se estiman 2 parámetros ($\alpha$ y $\beta$), como en una regresión logística. 

- Requiere pocos datos

- Es útil cuando el modelo no calibrado tiene errores similares en predicción de valores bajos y altos 

<br>
**Referencias:**

- Platt, John. (2000). Probabilistic Outputs for Support Vector Machines and Comparisons to Regularized Likelihood Methods. Adv. Large Margin Classif, Volume 10 (pp. 61-74).

:::

::: {style="font-size: 50%;"}
## Calibración sigmoide (Platt scaling) (Cont.)

In [ ]:
# | echo: true
# | eval: false
# | code-fold: true
# | code-summary: "Ver código"
pipe_sigmoid_hist, _, preds_hist = calibrate_model(
    pipe=pipe_hist,
    X_valid=X_valid, y_valid=y_valid, 
    X_test=X_test, y_test=y_test,
    cal_sigmoid=True,
    cal_isotonic=False,
    model_name=''
)
plot_calibration(
    preds=preds_hist,
    y_pred_prob_calibrated='pred_sigmoid',
    model_name=''
)

::: {.nav-pills}
::: {.panel-tabset}

In [ ]:
# | output: asis
for i in models_dict.keys():
    display(Markdown(f"## {i}"))

    # Curvas de calibración
    pipe_sigmoid, _, preds = calibrate_model(
        pipe=models_dict.get(i), 
        X_valid=X_valid, y_valid=y_valid, 
        X_test=X_test, y_test=y_test,
        cal_sigmoid=True,
        cal_isotonic=False,
        model_name=i
    )
    plt.show()

    # Calibración 
    plot_calibration(
        preds=preds,
        y_pred_prob_calibrated='pred_sigmoid',
        model_name=i
    )
    plt.title('Calibración')
    plt.show()

    display(Markdown(f" "))

:::
:::

In [ ]:
# Cálculo manual
# clf_log = LogisticRegression(random_state=42, C=0.1, solver="lbfgs")
# y_valid_pred_uncalibrated = pipe.predict_proba(X_valid)[:, 1]
# clf_log.fit(X=pd.DataFrame({"pred_prob": y_valid_pred_uncalibrated}), y=y_valid)

# preds["pred_sigmoid_manual"] = clf_log.predict_proba(preds[["pred_prob"]])[:, 1]

# Verificar por qué no funciona
# plt.figure()
# sns.scatterplot(
#     x="pred_prob",
#     y="pred_sigmoid_manual",
#     data=preds,
#     alpha=0.7,
#     label="Calibración manual",
# )

:::

::: {style="font-size: 50%;"}
## Calibración isotónica

Se ajusta un regresor isotónico no paramétrico, que produce una función escalonada no decreciente:

> $\sum_{i=1}^{n}(y_i - \hat{f_i})^2$
> 
> Sujeto a $\hat{f_i}$ >= $\hat{f_j}$ siempre $f_i$ > $f_j$ 
> $y_i$: etiqueta verdadera de la obseravción $i$ y sea la salida del clasificador calibrado para la muestra (es decir, la probabilidad calibrada)

In [ ]:
# print(f'% observado: {y_test.sum()/len(y_test):.3}')
# print(f'Prob promedio: {preds.pred_prob.mean():.3}')
# print(f'Prob promedio (calibración sigmoide): {prob_pos_sigmoid.mean():.3}')
# print(f'Prob promedio (calibración isotónica): {prob_pos_isotonic.mean():.3}')

:::

::: {style="font-size: 50%;"}
## Calibración isotónica (Cont.)

::: {.nav-pills}
::: {.panel-tabset}

In [ ]:
# | output: asis
for i in models_dict.keys():
    display(Markdown(f"## {i}"))

    pipe_sigmoid, pipe_isotonic, preds = calibrate_model(
        pipe=models_dict.get(i), 
        X_valid=X_valid, y_valid=y_valid, 
        X_test=X_test, y_test=y_test,
        cal_sigmoid=True,
        cal_isotonic=True,
        model_name=i
    )
    plt.show()

    plot_calibration(preds=preds, y_pred_prob_calibrated='pred_isotonic', model_name=i)
    plt.show()

    display(Markdown(f" "))

:::
:::
:::


::: {style="font-size: 50%;"}
## Tabla de calibración
::: {.nav-pills}
::: {.panel-tabset}

In [ ]:
# | output: asis
for i in models_dict.keys():
    display(Markdown(f"## {i}"))
    _, _, preds = calibrate_model(
        pipe=models_dict.get(i), 
        X_valid=X_valid, y_valid=y_valid, 
        X_test=X_test, y_test=y_test,
        cal_sigmoid=True,
        cal_isotonic=True,
        model_name=i,
        plot_cal=False
    )
    display(Markdown('Modelo no calibrado'))
    calibration_table(preds=preds, bins=5, title='Modelo no calibrado')
    display(Markdown('Modelo calibrado (sigmoide)'))
    calibration_table(preds=preds, pred_column='pred_sigmoid', bins=5, title='Modelo calibrado (sigmoide)')
    display(Markdown(f" "))

:::
:::
:::


::: {style="font-size: 50%;"}
## Comparativa de métricas

In [ ]:
metrics_df_sigmoid = pd.DataFrame()
metrics_df_isotonic = pd.DataFrame()
preds_dict = {}
for i in models_dict.keys():
    pipe = models_dict.get(i)
    _, _, preds = calibrate_model(
        pipe=models_dict.get(i), 
        X_valid=X_valid, y_valid=y_valid, 
        X_test=X_test, y_test=y_test,
        cal_sigmoid=True,
        cal_isotonic=True,
        model_name=i,
        plot_cal=False
    )
   
    m = calculate_clf_metrics(
        preds['y_obs'], preds['pred_class_sigmoid'], preds['pred_sigmoid'], 
        name=i
    ).reset_index().rename({'index':'Modelo'},axis=1)
    metrics_df_sigmoid = pd.concat([metrics_df_sigmoid, m], axis=0)

    m = calculate_clf_metrics(
        preds['y_obs'], preds['pred_class_sigmoid'], preds['pred_isotonic'], 
        name=i
    ).reset_index().rename({'index':'Modelo'},axis=1)
    metrics_df_isotonic = pd.concat([metrics_df_isotonic, m], axis=0)

::: {.nav-pills}
::: {.panel-tabset}

In [ ]:
# | output: asis
metrics_dfs = {
    'Modelo no calibrado': metrics_df,
    'Calibración sigmoide': metrics_df_sigmoid,
    'Calibración isotónica': metrics_df_isotonic
}
for i in metrics_dfs.keys():
    display(Markdown(f"## {i}"))
    display(GT(metrics_dfs.get(i).round(2))
        .data_color(
            columns=['Accuracy','Precision','Recall','F1','ROC AUC'],
            palette=[color_verde_claro, color_verde]
        )
        .data_color(
            columns=['Log Loss','Brier Loss'], palette=['white', 'red']
        )
    )
    display(Markdown(f" "))

:::
:::
:::


# Comentarios finales

::: {style="font-size: 50%;"}
## Comentarios finales

Se busca un modelo en donde la probabilidad promedio de cada bin se corresponda con el % de clase positiva observado en ese bin según las predicciones del modelo.

Esto es útil para la toma de decisiones, ya que permite establecer punto de cortes diferenciales en función de la aversión en riesgo de la entidad.

Por ejemplo:

- En un modelo de scoring crediticio, otorgarle créditos a N individuos con probabilidad en cierto intervalo tiene un riesgo asociado (mora esperada)
- En un modelo de churn (abandono) de clientes, otorgarle una promoción a individuos de cierto intervalo de probabilidad de abandono tiene un costo asociado (descuento para individuos que no abandonarían)
- Entre otros.

:::

::: {style="font-size: 50%;"}
## Referencias / Recursos

[Probability Calibration Workshop - PyData 2020](https://www.youtube.com/watch?v=A1NGGV3Z4m4&list=PLeVfk5xTWHYBw22D52etymvcpxey4QFIk&ab_channel=numeristical)

:::


## Contacto

{{< fa brands linkedin size=1x >}} [karinabartolome](https://www.linkedin.com/in/karinabartolome/)

{{< fa brands twitter size=1x >}} [@karbartolome](https://twitter.com/karbartolome)

{{< fa brands github size=1x >}} [@karbartolome](http://github.com/karbartolome)

{{< fa link >}} [Blog](https://karbartolome-blog.netlify.com)
